In [1]:
import os   #cette ligneet celle du dessous sonts sensées trouver la pathing du fichier csv qu'on veut ouvrir avec PANDAS 
os.getcwd()
%matplotlib inline
import numpy as np
import pandas as pd
import statsmodels.api as sm
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint
from enum import Enum
import collections, numpy # count the occurrence of certain item in an ndarray in Python
sns.set()

pd.options.display.max_rows = 10000

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Lecture de notre data pricipal :

data = pd.read_csv('data/EdStatsData.csv')


In [3]:
# Selection des data pour l'année 2015:

data = data.drop(['Country Name','Unnamed: 69'], axis=1)
data = data.drop(data.loc[:,'1970':'2014'], axis=1)
data = data.drop(data.loc[:,'2016':'2100'], axis=1)
data = pd.melt(data,id_vars = ['Indicator Code','Indicator Name','Country Code',],var_name='Year')

# on drop seulement un seul indicateur d'un pays et pas tout les indics du pays 
data = data.dropna()

data.sample(5)



,Indicator Code,Indicator Name,Country Code,Year,value
350521,SP.POP.AG01.TO.UN,"Population, age 1, total",GMB,2015,57644.000000
482449,SP.SEC.UTOT.MA.IN,Population of the official age for upper secon...,KGZ,2015,108048.000000
808824,SP.POP.0713.MA.UN,"Population, ages 7-13, male",TON,2015,8345.000000
527575,PRJ.ATT.25UP.2.FE,Wittgenstein Projection: Percentage of the pop...,MDG,2015,0.080000
819411,LO.PIAAC.NUM.P90,PIAAC: Distribution of Adult Numeracy Scores: ...,TUR,2015,286.725297


In [4]:
# visualisation des indicateurs 

# data['Indicator Name'].value_counts().sort_values(ascending=False)

In [5]:
# L'idée ici c'est de prendre les indicateurs qui se sont repeter au moins 70% du nombres de pays disponible ( dans notre cas 242)

data=data.groupby('Indicator Code').filter(lambda x: x['Indicator Code'].size>130)

# Je selectionne les indicateurs interessants et qui ont la fréquence d'apparition la plus elevée(voir en haut)

test = data.loc[data['Indicator Code'].isin(['SP.POP.TOTL', 'IT.NET.USER.P2','SP.SEC.TOTL.IN',
                                         'NY.GDP.PCAP.CD','SP.SEC.TOTL.IN',
                                         'SP.POP.1524.TO.UN','PRJ.POP.ALL.3.MF','PRJ.POP.ALL.4.MF'])]

# j'ai visualiser avec un value counts les pays et j'enleve ceux qui se sont repetés moins6 fois car jai 8 indics pricipaux


test = test.groupby(['Country Code']).filter(lambda x : x['Country Code'].size>6)

test.sort_values('Country Code').sample(5)


,Indicator Code,Indicator Name,Country Code,Year,value
823493,SP.POP.TOTL,"Population, total",TKM,2015,5.565284e+06
684013,SP.SEC.TOTL.IN,Population of the official age for secondary e...,RWA,2015,1.547792e+06
145258,SP.SEC.TOTL.IN,Population of the official age for secondary e...,BHR,2015,9.052500e+04
818670,IT.NET.USER.P2,Internet users (per 100 people),TUR,2015,5.374498e+01
287115,NY.GDP.PCAP.CD,GDP per capita (current US$),DNK,2015,5.301300e+04


In [26]:
# j'ai mis tout les 8 indicateurs comme colonnes pour mieux visualiser chaque pays dans UNE SEULE ligne, donc on a toute les infos a chaque ligne

test2 = test.pivot('Country Code',columns='Indicator Name', values='value')
test2 = test2.reset_index()

# Vu qu'il ya trés peu de NaN a ce point j'ai decider de fill tout les NaN car ils ne vont pas impacter mes conclusions 
# test2.apply(lambda x: x.fillna(method = 'ffill',inplace=True))

# j'ai filtrer par personnnes ayant l'accés a internet j'ai prix un minimum de 70%
test2 = test2.loc[test2['Internet users (per 100 people)']>= 70] 
test2 = test2.loc[test2['Population, ages 15-24, total']>= 400_000] 

test2.head()

Indicator Name,Country Code,GDP per capita (current US$),Internet users (per 100 people),"Population of the official age for secondary education, both sexes (number)","Population, ages 15-24, total","Population, total",Wittgenstein Projection: Population in thousands by highest level of educational attainment. Post Secondary. Total,Wittgenstein Projection: Population in thousands by highest level of educational attainment. Upper Secondary. Total
1,ARE,39101.746890,90.500000,525697.0,743608.0,9154302.0,1315.87,2768.80
4,AUS,56554.038761,84.560519,1714038.0,2914620.0,23789338.0,6276.89,8528.74
5,AUT,44255.583356,83.940142,690920.0,964629.0,8633169.0,1817.25,3593.74
6,AZE,5500.310382,77.000000,907062.0,1544146.0,9649341.0,1110.37,4804.83
8,BEL,40375.388639,85.052900,730126.0,1241946.0,11274196.0,3107.79,2973.51


In [27]:
# je veux voir a partir de quel GDP on considére un pays comme potentiel 

test2.groupby('Country Code')['Population, ages 15-24, total'].mean().sort_values()


Country Code
LTU      411242.0
KWT      463706.0
IRL      580379.0
NZL      615099.0
FIN      639391.0
SVK      646952.0
NOR      646961.0
DNK      710024.0
ARE      743608.0
LBN      774973.0
HKG      856743.0
CHE      959002.0
AUT      964629.0
CZE     1063674.0
HUN     1123703.0
SWE     1134811.0
ISR     1211599.0
BEL     1241946.0
AZE     1544146.0
NLD     2005912.0
KAZ     2393020.0
AUS     2914620.0
ESP     4285743.0
CAN     4373511.0
MYS     5332642.0
KOR     6456561.0
FRA     7567872.0
GBR     7731522.0
DEU     8682394.0
JPN    12157426.0
RUS    14661984.0
USA    45147517.0
Name: Population, ages 15-24, total, dtype: float64

In [25]:
# sns.barplot(x="Country Code",y="Population of the official age for secondary education, both sexes (number)",data=test2)

# sns.distplot(test2['Population, ages 15-24, total'],kde=False)


In [ ]:
# sns.stripplot(x = 'GDP per capita (current US$)',y = 'Internet users (per 100 people)',data = test2)



In [ ]:
# sns.stripplot(y = 'GDP per capita (current US$)',x = 'Enrolment in secondary general, both sexes (number)',data = test2)